<a href="https://colab.research.google.com/github/akatakan/QLoRa/blob/main/QLoRA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q -U bitsandbytes
!pip install -q -U transformers
!pip install -q -U peft
!pip install -q -U accelerate
!pip install -q datasets
!pip install -q trl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 39.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.2/245.2 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.4/102.4 kB 14.8 MB/s eta 0:00:00


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, BitsAndBytesConfig,DataCollatorForLanguageModeling
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model
from huggingface_hub import notebook_login
from datasets import load_dataset, DatasetDict
import torch
from trl import SFTTrainer

In [ ]:
from huggingface_hub import login
from google.colab import userdata
login(userdata.get('hf'))

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
base_model = "meta-llama/Meta-Llama-3-8B-Instruct"

# NF4 = normalize float 4 | pure FP4 , quantization çeşitleri
# Quantizationda ağıtlıklar 4 bit olarak saklanırken, hesaplamaları 16 veya 32 bit veri tipleri olarak ayarlanabilir
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True, #ikinci bir quantization kullanmamızı sağlar, bu da parametre başına 0.4 bitlik ek tasarrufluk sağlar
    bnb_4bit_quant_type='nf4',
    bnb_4bit_compute_dtype=torch.bfloat16 #daha esnek, daha fazla fine tuning
)



In [ ]:
tokenizer = AutoTokenizer.from_pretrained(base_model)
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config = bnb_config,
    device_map="auto"
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


...

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

In [ ]:
trained_model_id = "Llama-3-8B-turkishQA"
output_dir = './results/' + trained_model_id


In [ ]:
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [ ]:
import bitsandbytes as bnb
def find_all_linear_names(bits, model):
    cls = bnb.nn.Linear4bit if bits == 4 else (bnb.nn.Linear8bitLt if bits == 8 else torch.nn.Linear)
    lora_module_names = set()
    for name, module in model.named_modules():
        if isinstance(module, cls):
            names = name.split('.')
            lora_module_names.add(names[0] if len(names) == 1 else names[-1])


    if 'lm_head' in lora_module_names: # needed for 16-bit
        lora_module_names.remove('lm_head')
    return list(lora_module_names)

modules = find_all_linear_names(4,model)
modules

['o_proj', 'q_proj', 'k_proj', 'v_proj', 'down_proj', 'up_proj', 'gate_proj']

In [ ]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [ ]:
from peft import LoraConfig

peft_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=modules,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, peft_config)
print_trainable_parameters(model)

trainable params: 20971520 || all params: 4561571840 || trainable%: 0.4597432800707574


In [ ]:
dataset = "Yudum/turkish-instruct-dataset"

In [ ]:
train_dataset = load_dataset(dataset,split="train")

Generating train split:   0%|          | 0/82353 [00:00<?, ? examples/s]

In [ ]:
def generate_prompt(data_point):
    if data_point["input"]:
        return f"""Aşağıda, daha geniş bir bağlam sağlayan girdiyle birlikte bir görevi açıklayan talimat bulunmaktadır. Talimatı yeterince sağlayan bir çıktı yaz.

                ### Talimat:
                {data_point["instruction"]}

                ### Girdi:
                {data_point["input"]}

                ### Çıktı:
                {data_point["output"]}"""
    else:
      return f"""Aşağıda bir görevi açıklayan talimat bulunmaktadır. Talimatı yeterince sağlayan bir çıktı yaz.

              ### Talimat:
              {data_point["instruction"]}

              ### Çıktı:
              {data_point["output"]}"""

EOS_TOKEN = tokenizer.eos_token

In [ ]:
def formatting_func(examples):
  texts = []
  text = generate_prompt(examples) + EOS_TOKEN
  texts.append(text)
  return { "text": texts,}

In [ ]:
train_dataset = train_dataset.map(lambda x: print(x), batched = True)

Map:   0%|          | 0/82353 [00:00<?, ? examples/s]

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub data rate 

{'input': ['Konu: Üründen hayal kırıklığına uğradım\r\n\r\nSayın Destek Ekibi,\r\n\r\nYakın zamanda web sitenizden bir ekmek kızartma makinesi satın aldım ve performansından memnun değilim. Her kullandığımda ekmeğim yanıyor. Ekmek kızartma makinesini iade edip para iadesi almak istiyorum.\r\n\r\nSamimi olarak,\r\nHayal kırıklığına uğramış müşteri', '', 'Günümüzün rekabetçi iş piyasasında, büyük kariyer hamlelerini gerçekleştirmek için kişinin öne çıkması gerekir. Başarılı olmak istiyorsanız, hem işinize yaklaşımınız hem de meslektaşlarınızla iletişim şekliniz açısından işverenler üzerinde olumlu bir izlenim bırakmak önemlidir.', '', '', "Apple Inc., yeni iPhone 13'ün piyasaya sürüldüğünü duyurdu. A15 Biyonik çipi, geliştirilmiş 5G özellikleri ve şık tasarımıyla iPhone 13, akıllı telefon sektöründe devrim yaratmaya hazırlanıyor. Apple'ın en yeni ürünü, benzersiz bir kullanıcı deneyimi sunarak şirketin teknolojik inovasyondaki lider konumunu güçlendiriyor.", '', 'Selam! Sizlere yeni prog

In [ ]:
train_dataset
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
training_arguments = TrainingArguments(
          fp16= not torch.cuda.is_bf16_supported(),
          bf16= torch.cuda.is_bf16_supported(),
          gradient_accumulation_steps=8,
          gradient_checkpointing=True,
          learning_rate=2.0e-05,
          log_level="info",
          logging_steps=5,
          lr_scheduler_type="cosine",
          max_steps=100,
          output_dir=output_dir,
          overwrite_output_dir=True,
          per_device_train_batch_size=8,
          push_to_hub=True,
          hub_model_id=trained_model_id,
          save_total_limit=1,
          load_best_model_at_end=True
          seed=42,
)

In [ ]:
torch.cuda.empty_cache()

In [ ]:
trainer = SFTTrainer(
        model=model,
        args=training_arguments,
        train_dataset=train_dataset,
        dataset_text_field = "text",
        formatting_func = formatting_func,
        tokenizer=tokenizer,
        peft_config=peft_config,
        max_seq_length= 2048
)

Map:   0%|          | 0/82353 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs
Using auto half precision backend


In [ ]:
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.748 GB.
9.449 GB of memory reserved.


In [ ]:
model.config.use_cache = False
trainer.train()

***** Running training *****
  Num examples = 82,353
  Num Epochs = 1
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 128
  Gradient Accumulation steps = 8
  Total optimization steps = 100
  Number of trainable parameters = 20,971,520


OutOfMemoryError: CUDA out of memory. Tried to allocate 380.00 MiB. GPU 0 has a total capacity of 14.75 GiB of which 279.06 MiB is free. Process 3304 has 14.47 GiB memory in use. Of the allocated memory 13.71 GiB is allocated by PyTorch, and 651.21 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:

used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")